# Uniendo los excels para crear una sola base de datos

### Observaciones diarias de precios entre mayo 2023 y enero 2015. 
### No se tienen en cuenta festivos ni fines de semana lo cual debe ser reemplazado 


In [85]:
import pandas
import numpy
import os 


In [86]:
ejemplo_1 = pandas.read_excel("Archivos_raw/anex_abr_01_2022.xlsx")
x = ejemplo_1.iloc[0,0]
z = ejemplo_1.iloc[:,0]
y = (ejemplo_1.iloc[:,0] == "Precio $/Kg").idxmax()

ejemplo_1 = ejemplo_1.iloc[y:]
ejemplo_1.columns = ejemplo_1.iloc[0]
ejemplo_1 = ejemplo_1.iloc[1:]

ejemplo_1 = ejemplo_1.drop(columns=numpy.nan)

ejemplo_1['Date'] = x

In [87]:
ejemplo_1.iloc[:,0]
valores_interes = ejemplo_1['Precio $/Kg']
valores_interes = valores_interes[2:]
valores_interes = valores_interes[valores_interes != 'Frutas frescas']
valores_interes = valores_interes[valores_interes != 'Var%: Variación porcentual con respecto al promedio del día de mercado anterior de la misma plaza']
valores_interes = valores_interes[valores_interes != '*Variedad predominante en el mercado']
valores_interes = valores_interes[valores_interes != 'n.d. : no disponible']

In [88]:
ejemplo_1 = ejemplo_1[ejemplo_1['Precio $/Kg'].isin(valores_interes)]

In [89]:
directorio = "Archivos_raw/"
file_path = []
for filename in os.listdir(directorio):
    file_path.append(os.path.join(directorio, filename))

### Proximamente prueba #1 de merge de los reportes en un solo dataframe

In [90]:
df = pandas.DataFrame()
error = []
for i in file_path:
    try:
        data = pandas.read_excel(i)
        x = data.iloc[0,0]
        z = data.iloc[:,0]
        y = (data.iloc[:,0] == "Precio $/Kg").idxmax()

        data = data.iloc[y:]
        data.columns = data.iloc[0]
        data = data.iloc[1:]

        data = data.drop(columns=numpy.nan)

        data['Date'] = x   
        data = data[data['Precio $/Kg'].isin(valores_interes)]

        df = pandas.concat([df, data])

        if x == "Miércoles 09 de 10 de 2019":
            error.append(i)

        else:
            continue

    except:
        print(i)
        error.append(i)
    

Archivos_raw/anex_mar_10_2022.xlsx
Archivos_raw/mayoristas_noviembre_16_2021.xlsx
Archivos_raw/mayoristas_noviembre_17_2021.xlsx
Archivos_raw/mayoristas_noviembre_18_2021.xlsx
Archivos_raw/mayoristas_noviembre_19_2021.xlsx
Archivos_raw/mayoristas_noviembre_22_2021.xlsx
Archivos_raw/mayoristas_noviembre_23_2021.xlsx
Archivos_raw/mayoristas_noviembre_25_2021.xlsx
Archivos_raw/mayoristas_noviembre_30_2021.xlsx


In [91]:
fechas = df['Date'].unique()

# De todos los Archivos se identifiaron 11 que tienen errores 

#### Existen 11 archivos que presentan errores al procesarlos conjuntamente. Se revisaran 1-1

In [92]:
data_error = pandas.read_excel(error[0])
x = data_error.iloc[0,0]
z = data_error.iloc[:,0]
y = (data_error.iloc[:,0] == "Precio $/Kg").idxmax()

data_error = data_error.iloc[y:]
data_error.columns = data_error.iloc[0]
data_error = data_error.iloc[1:]

data_error = data_error.drop(columns=numpy.nan)

data_error['Date'] = x

data_error = data_error[data_error['Precio $/Kg'].isin(valores_interes)]

df = df.reset_index(drop = True)
data_error = df.reset_index(drop=True)

FinalDatabase = pandas.concat([df,data_error], ignore_index=True)


##### El 02 de octubre de 2020 se excluye por error en el reporte
##### Por diferencias en la forma del informe se excluyen el resto de errores del archivo

In [93]:
FinalDatabase.to_csv("Final_data.csv")